### Downlaod images from the urls given in the txt file, code is update for the num of txt files inside folder
### Save the images with the same name of the txt file

In [ ]:
import os
import requests
from tqdm import tqdm

# === CONFIGURATION ===
# txt_path = "C:/Users/LT/Downloads/original_images_url_project_3964 (1).txt"     # Path to your text file
txt_folder = "C:/Users/LT/Downloads/original_images_url_project_group_112/media/images"
output_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb"     # Where to save images
timeout_sec = 20                     # Timeout for each request (seconds)

# === CREATE OUTPUT FOLDER ===
os.makedirs(output_dir, exist_ok=True)

txt_files = [f for f in os.listdir(txt_folder) if f.endswith(".txt")]

if not txt_files:
    print("❌ No .txt files found in the folder.")

for txt_file in txt_files:

    txt_path = os.path.join(txt_folder, txt_file)
    base_name = os.path.splitext(txt_file)[0]
    save_folder = os.path.join(output_dir, base_name)

    os.makedirs(save_folder, exist_ok=True)
# === READ ALL URLS ===
    with open(txt_path, "r", encoding="utf-8") as f:
        urls = [line.strip().strip('"') for line in f if line.strip()]

    print(f"🧾 Found {len(urls)} image URLs in {txt_path}")

    # === DOWNLOAD LOOP ===
    for url in tqdm(urls, desc="Downloading images"):
        # Extract filename (remove query parameters)
        filename = os.path.basename(url.split("?")[0])
        # save_path = os.path.join(output_dir, filename)
        save_path = os.path.join(save_folder, filename)

        # Skip if already downloaded
        if os.path.exists(save_path):
            continue

        try:
            response = requests.get(url, stream=True, timeout=timeout_sec)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    for chunk in response.iter_content(8192):
                        f.write(chunk)
            else:
                print(f"⚠️ Failed {filename}: HTTP {response.status_code}")
        except Exception as e:
            print(f"❌ Error downloading {filename}: {e}")

    print(f"✅ Finished '{txt_file}' — Saved in: {save_folder}\n")

print("\n🎉 All downloads complete!")

# print("\n✅ Download complete! Check your 'downloaded_images' folder.")


### Rename the folders according to the name of the images inside the folder
### if all the images are of same name then it'll change the folder name otherwise skip that frame

In [ ]:
import os
import re

# === CONFIGURATION ===
root_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb"  # Change this to your root folder

# === FUNCTION TO EXTRACT PREFIX (before last underscore) ===
def get_prefix(filename):
    match = re.match(r"^(.*)_[0-9]+", os.path.splitext(filename)[0])
    return match.group(1) if match else None

# === MAIN LOOP ===
for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Collect all valid image filenames
    images = [f for f in os.listdir(folder_path)
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    if not images:
        print(f"⚠️ No images found in {folder_name}")
        continue

    # Extract prefixes
    prefixes = {get_prefix(img) for img in images if get_prefix(img)}

    if len(prefixes) == 1:
        prefix = list(prefixes)[0]
        new_folder_name = os.path.basename(prefix)  # e.g. c1_v3
        new_folder_path = os.path.join(root_dir, new_folder_name)

        # Avoid overwriting existing folders
        if new_folder_path == folder_path:
            print(f"✅ Already named correctly: {folder_name}")
        elif os.path.exists(new_folder_path):
            print(f"⚠️ Skipped: {new_folder_name} already exists.")
        else:
            os.rename(folder_path, new_folder_path)
            print(f"✅ Renamed '{folder_name}' → '{new_folder_name}'")
    else:
        print(f"❌ Inconsistent prefixes in {folder_name}: {prefixes}")

print("\n🎯 Folder renaming complete!")


### Extract the labels files from the zip file and place them in the respective folder

In [ ]:
import os
import zipfile
import shutil

# === CONFIGURATION ===
main_zip_dir = "C:/Users/LT/Downloads/project_group_112_yolo_v5/media/images"   # Folder containing inner ZIPs like Qiyas_c1_v3_export_YOLO_V5.zip
target_base_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb"  # Folder containing c1_v3, c1_v4, etc.
temp_extract_dir = "C:/wajahat/hand_in_pocket/dataset/temp_extract"  # Temporary folder for extraction

# === ENSURE TEMP FOLDER EXISTS ===
os.makedirs(temp_extract_dir, exist_ok=True)

# === LOOP THROUGH ALL INNER ZIPS ===
for zip_name in os.listdir(main_zip_dir):
    if not zip_name.endswith(".zip"):
        continue

    zip_path = os.path.join(main_zip_dir, zip_name)
    print(f"\n📦 Processing {zip_name}")

    # Extract camera/video name (e.g. c1_v3)
    try:
        base_name = zip_name.split("_export_")[0].replace("Qiyas-", "")
    except Exception:
        print(f"⚠️ Could not parse name from {zip_name}, skipping.")
        continue

    target_dir = os.path.join(target_base_dir, base_name)
    # target_folder = os.path.join(target_base_dir, base_name)
    # target_dir = os.path.join(target_folder, "labels")
    # print(target_dir)

    if not os.path.exists(target_dir):
        print(f"❌ Target folder not found: {target_dir}")
        continue

    # os.makedirs(target_dir, exist_ok=True)

    # === Extract inner zip ===
    with zipfile.ZipFile(zip_path, 'r') as inner_zip:
        inner_zip.extractall(temp_extract_dir)

    # === Find .txt and .label files ===
    files_to_move = []
    for root, _, files in os.walk(temp_extract_dir):
        for file in files:
            if file.endswith(".txt") or file.endswith(".labels"):
                files_to_move.append(os.path.join(root, file))

    print(f"🧾 Found {len(files_to_move)} files (.txt + .label) to move")

    # === Move files to target folder ===
    for src_path in files_to_move:
        dest_path = os.path.join(target_dir, os.path.basename(src_path))
        try:
            shutil.move(src_path, dest_path)
        except Exception as e:
            print(f"⚠️ Could not move {src_path}: {e}")

    # === Cleanup temp folder ===
    shutil.rmtree(temp_extract_dir)
    os.makedirs(temp_extract_dir, exist_ok=True)

    print(f"✅ Moved all files to {target_dir}")

print("\n🎯 All zips processed successfully!")


### Move the labels and images in two folders labels and images

In [ ]:
import os
import shutil

# === CONFIGURATION ===
main_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb"  # path to your main directory

# Define which file extensions are considered images or labels
image_exts = {".jpg", ".jpeg", ".png"}
label_exts = {".txt", ".label"}

# === LOOP THROUGH ALL SUBFOLDERS ===
for folder_name in os.listdir(main_dir):
    folder_path = os.path.join(main_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue  # skip non-folder items

    print(f"\n📁 Processing folder: {folder_name}")

    images_dir = os.path.join(folder_path, "images")
    labels_dir = os.path.join(folder_path, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    # Loop through all files in the current folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Skip subfolders (images/labels we just created)
        if os.path.isdir(file_path):
            continue

        ext = os.path.splitext(file_name)[1].lower()

        # Move image files
        if ext in image_exts:
            shutil.move(file_path, os.path.join(images_dir, file_name))
            print(f"🖼️ Moved image → {images_dir}")

        # Move label files
        elif ext in label_exts:
            shutil.move(file_path, os.path.join(labels_dir, file_name))
            print(f"🏷️ Moved label → {labels_dir}")

print("\n✅ Sorting complete! All folders processed successfully.")

### Show the bounding box without labels with the same color

In [ ]:
import os
import cv2

# === CONFIGURATION ==="C:\wajahat\hand_in_pocket\dataset\images_bb\c1_v3\images"
images_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb/c1_v3/images"
labels_dir = "C:/wajahat/hand_in_pocket/dataset/images_bb/c1_v3/labels"
class_file = "C:/wajahat/hand_in_pocket/dataset/images_bb/_darknet.labels"  # your .labels file

# === LOAD CLASS NAMES ===
if os.path.exists(class_file):
    with open(class_file, "r") as f:
        class_names = [line.strip() for line in f if line.strip()]
else:
    class_names = []
    print("⚠️ No class file found — labels will be shown as class IDs")

# === VISUALIZATION LOOP ===
for img_name in sorted(os.listdir(images_dir)):
    if not img_name.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(images_dir, img_name)

    # Derive label file name (you said format is: c1_v3_c1_v3_00.txt for c1_v3_00.jpg)
    label_name = f"c1_v3_{os.path.splitext(img_name)[0]}.txt"
    label_path = os.path.join(labels_dir, label_name)

    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read image: {img_path}")
        continue
    h, w = img.shape[:2]

    # Read YOLO label file
    if not os.path.exists(label_path):
        print(f"⚠️ No label found for {img_name}")
    else:
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls_id, x_center, y_center, bw, bh = map(float, parts)
                cls_id = int(cls_id)

                # Convert normalized to pixel coordinates
                x_center *= w
                y_center *= h
                bw *= w
                bh *= h
                x1 = int(x_center - bw / 2)
                y1 = int(y_center - bh / 2)
                x2 = int(x_center + bw / 2)
                y2 = int(y_center + bh / 2)

                # Draw box
                color = (0, 255, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                # Draw label text
                label_text = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                cv2.putText(img, label_text, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2, cv2.LINE_AA)

    # Show image
    cv2.imshow("YOLO Annotation Viewer", img)
    key = cv2.waitKey(0)  # press any key to go to next image
    if key == 27:  # ESC to exit early
        break

cv2.destroyAllWindows()
print("✅ Visualization complete!")
